In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('vader_lexicon')

import tweepy
from kafka import KafkaProducer
from time import sleep
import json 
import pandas as pd

C:\Users\selma\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\selma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\selma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\selma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def preprocessing(text, lang): #removing useless information in the tweets that won't serve in sentiment analysis
    text = text.lower()
    text = re.sub("https?://[\S]+", "", text) #remove links
    text = re.sub(r"@[\S]+", "", text) #remove tags
    text = re.sub("[^a-zA-Z]", " ", text) #remove numbers 
    text = word_tokenize(text) #tokenize the text
    text = [word for word in text if ((word not in stopwords.words(lang)) and (word.upper()!='RT'))] #remove stop words
    text = " ".join(word for word in text)
    return text

In [3]:
column_names=["lang","possibly_sensitive",'text', 'created_at', 'label']
df = pd.DataFrame(columns = column_names)
counter = 1 # to debug the message number
topic_name = 'project-tweets'
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGrckgEAAAAARz1xWFSBVNs36y9ceyP8ecUmA4o%3DSwhksbSUmEuL3oqyygYu0lvJyQw4CVyWa0sow8hTbiNLOwYE16')
# searching tweets about covid
query = 'ukraine'
producer = KafkaProducer(bootstrap_servers="localhost:9092", value_serializer=lambda x: json.dumps(x).encode("utf8"),)

print(producer.bootstrap_connected()) 

True


In [4]:
while (counter <=1000):
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
    tweet_fields=[ 'created_at', 'lang', 'possibly_sensitive'], max_results=30).flatten():
        if tweet["lang"]=='en':
            lang='english'
        elif tweet["lang"]=='fr':
            lang='french'
        else:
            lang =''
        
        if lang !='':
            msg= preprocessing(tweet['text'], lang)    
            y=0 #neutral by default
            score = SentimentIntensityAnalyzer().polarity_scores(msg)
            #labelling data:
            if score['neg'] > score['pos']:
                y = -1 #-1 means negative
            elif score['pos'] > score['neg']:
                y= 1 #+1 means positive sentiment

            message = {
                    "lang": tweet["lang"],
                    "possibly_sensitive": tweet["possibly_sensitive"],
                    "text": msg,
                    "created_at": tweet["created_at"].strftime("%Y-%m-%d %H:%M:%S"),
                    "label" : y,
                }
            print(message['label'])
            print(msg)
            print('\n')
            if msg!='':
                df = df.append(message, ignore_index=True)
                producer.send(topic_name,message)
                print('sent message nbr:',counter)
                counter+=1
    sleep(1)

-1
russia terror attack today let give incremental sanctions amp go one set western com




C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\A

sent message nbr: 1
-1
would normally front page news identical terrorist attacks around world condemned every gove


sent message nbr: 2
1
gaza ukraine anyone care


sent message nbr: 3
1
classified documents found joe biden private office contained us intelligence materials related ukraine


sent message nbr: 4
0
wholesale gas prices russia invaded ukraine tell us still paying thousa


sent message nbr: 5
1
mudryk like love arsenal agreeing k per week wages k shaktar bullion van fc arrived ukra


sent message nbr: 6
1
ukraine humiliated western propagandists defense minister admitted nato proxy


sent message nbr: 7
-1
western nations sending billions ukraine supply weapons pays benefits


sent message nbr: 8
1
russians celebrate strike civilian house dnipro negotiations forgiveness arm ukraine


sent message nbr: 9
1
ok maybe ukraine believe tank know


sent message nbr: 10
-1
telling joe biden stealing highly classified documents ukraine stashing garage


sent message nbr: 11
-1
we

C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\A

sent message nbr: 15
-1
worry russia launched half kalibr missiles intended air raid ukraine whole dif


sent message nbr: 16
0
week germany handed ukraine another batch military aid includes armored demining vehicles mobile


sent message nbr: 17
-1
v drine admirateur perdu poutine a ordonn destruction ukraine bombarde


sent message nbr: 18
-1
british intelligence update ukraine war today jan latest war map kherson donetsk bakhmut oblasts


sent message nbr: 19
1
school peace today missile attack ukraine went shelter


sent message nbr: 20
-1
german armsmaker rheinmetall could deliver repaired leopard battle tanks ukraine earliest would need confirmed order begin repairs chief executive quoted saying bild newspaper sunday ukrainewar germany




...

-1
attacks dnipro elsewhere ukraine russia proven terrorist state call allies


sent message nbr: 1059
1
peace ukraine


sent message nbr: 1060
-1
ukraine drone germany chemical operations yemen weapons uk uranium syria killing protesters iraq government intervention affairs iran shooting ps flight killing imprisoning protesters charlie hebdo threatening attack irgcterrorists


sent message nbr: 1061
0
ukraine thooo


sent message nbr: 1062
1
gaza ukraine anyone care


sent message nbr: 1063
-1
anyone else ukraine thinking taking arms russians even knowing risks without combat experience nearly months war right fighting dying fear nuclear war


sent message nbr: 1064
-1
ukraine even developed anti ship weapon neptune based soviet rocket designs target russian flee


sent message nbr: 1065
0
visez peu barque usa hollande guerre ukraine va falloir tweeter corr




C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\A

sent message nbr: 1066
-1
people ukraine weapons need order fight back look ways help faster n


sent message nbr: 1067
0
mois macron donne millions rwanda num rique millions pakistan millions ukraine milli


sent message nbr: 1068
-1
never heard oh boy pull say called president ukraine ignorant


sent message nbr: 1069
-1
appalled russia attacks residential buildings dnipro ukraine condemn violence strongest


sent message nbr: 1070
-1
instead fighting russia ukraine america bombing hell klaus schwab davos right


sent message nbr: 1071
1
hi friends learn webdesign today gigs like tommytiern




C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\AppData\Local\Temp\ipykernel_18732\1838094768.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
C:\Users\selma\A

KeyboardInterrupt: 

In [5]:
#To save the data locally as a csv format
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
df.to_csv('data.zip', index=False,
          compression=compression_opts)